# Specific Test III. Learning Mass of Dark Matter Halo

Task: Using the provided dataset implement a regression algorithm to learn the mapping between lensing images and the lensing dark matter halo mass. You can use the machine learning algorithm of your choice. Please implement your approach in PyTorch and discuss your strategy.

## 

### DataSet Pre-Processing

### Importing Libraries and setup the device

In [21]:
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
device = torch.device("cpu")
path = 'F:/mini projet/dataLR2'

### Pre-Processing Class

This class contains the important methods for data loading, converting to tensors, and visualization

In [22]:
class Data():
    def __init__(self):
        super(Data, self).__init__()
        print("DONE")
        
    def loadData(self):
        listdirectory = os.listdir(path=path)
        dataLR = np.load(os.path.join(path, listdirectory[0]), allow_pickle=True)
        return dataLR
        
    def splittingData(self):
        frames = []
        labels = []
        for count, filename in enumerate(os.listdir(path)): 
            data = np.load(path+'/'+filename,allow_pickle=True)
            frames.append(data[:1][0])  
            labels.append(data[1:2][0]) 
        return frames,labels  
          
    def converting(self):
        frames,labels = obj.splittingData()
        frames = np.array(frames)
        labels = np.array(labels)
        return frames/255.0, labels
    
    def tesTrainSplit(self):
        X,y = self.converting()
        fr_tr, fr_te, lab_tr, lab_te = train_test_split(X, y, test_size=0.1, random_state=9)
        fr_tr = torch.tensor(fr_tr)
        fr_te = torch.tensor(fr_te)
        lab_tr = torch.tensor(lab_tr)
        lab_te = torch.tensor(lab_te)
        fr_tr = torch.unsqueeze(fr_tr, 1)
        fr_te = torch.unsqueeze(fr_te, 1)
        return fr_tr,fr_te,lab_tr,lab_te
    
    def tensorData(self):
        fr_tr,fr_te,lab_tr,lab_te = self.tesTrainSplit()
        train = torch.utils.data.TensorDataset(fr_tr, lab_tr)
        test = torch.utils.data.TensorDataset(fr_te, lab_te)
        train_loader = torch.utils.data.DataLoader(train, batch_size=64)
        test_loader = torch.utils.data.DataLoader(test, batch_size=64)
        return train_loader, test_loader

obj = Data()
data = obj.loadData()
print(data)

DONE
[array([[34, 29, 25, ..., 32, 31, 35],
       [36, 30, 39, ..., 31, 30, 39],
       [33, 36, 29, ..., 32, 27, 27],
       ...,
       [29, 27, 31, ..., 33, 27, 35],
       [39, 36, 25, ..., 22, 20, 29],
       [34, 19, 23, ..., 29, 25, 18]], dtype=int64)
 237.59861225420505]


In [23]:
frames,labels = obj.splittingData()

In [24]:
fr_tr,fr_te,lab_tr,lab_te = obj.tesTrainSplit()

After this, i will create the loader

In [26]:
train_loader, test_loader = obj.tensorData()

### Training 

As you can see here, im defining the following architecture for my model

Conv 2D ==> Conv 2D ==> Conv 2D ==> Batch Normalization ==> Max Pooling ==> Fully Connected ==> FC => Output


In [62]:
#Copirights Pytorch Official Website
class Regress(nn.Module):
    def __init__(self):
        super(Regress, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.conv3 = nn.Conv2d(32, 32, 3)
        self.bn = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(3, 2, padding=1)
        self.fc1 = nn.Linear(32*36*36, 500)
        self.fc2 = nn.Linear(500, 500)
        self.fc3 = nn.Linear(500, 1)
           
    def forward(self, x):
        x = self.bn(self.pool(F.relu(self.conv1(x))))
        x = self.bn(self.pool(F.relu(self.conv2(x))))
        x = self.bn(self.pool(F.relu(self.conv3(x))))
        x = x.view(-1, 32*36*36)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

model = Regress().to(device)


In [58]:
metric = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
epochs = 32

The following function for evaluating our model and training

In [59]:
def evaluate(data_loader):
    model.eval()
    loss = 0
    correct = 0
    
    for data, target in data_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        output = model(data)
        
        loss += F.cross_entropy(output, target, size_average=False).data[0]

        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    loss /= len(data_loader.dataset)
        
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))

In [60]:
#Copyrights Kaggle
def train():
    inputs = inputs.to(device)
    labels = labels.to(device).view(labels.shape[0], 1)
    outputs = model(inputs.float())
    loss = metric(outputs, labels.float())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    loop.set_description(f"Epoch [{epoch+1}/{epochs}]")
    loop.set_postfix(loss = running_loss/len(train_loader))

Start Training with 32 epochs

In [ ]:
model.train()
train_losses = []
for epoch in range(epochs):
    running_loss = 0
    loop = tqdm(train_loader, position=0)
    for inputs, label in loop:
        inputs = inputs.to(device)
        label = label.to(device).view(label.shape[0], 1)
        outputs = model(inputs.float())
        loss = metric(outputs, labels.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        loop.set_description(f"Epoch [{epoch+1}/{epochs}]")
        loop.set_postfix(loss = running_loss/len(train_loader))
    train_losses.append(running_loss/len(train_loader))

The following Method for making predictions


In [ ]:
def prediciton(data_loader):
    model.eval()
    test_pred = torch.LongTensor()
    
    for i, data in enumerate(data_loader):
        data = Variable(data, volatile=True)
        if torch.cuda.is_available():
            data = data.cuda()
            
        output = model(data)
        
        pred = output.cpu().data.max(1, keepdim=True)[1]
        test_pred = torch.cat((test_pred, pred), dim=0)
        
    return test_pred